In [152]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers

import tensorflow_hub as hub

from  IPython import display
from matplotlib import pyplot as plt

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [153]:
train_limpio = pd.read_csv("train_limpio_con_stemming.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad,the,...,he is,insultos_por_tweet,expresiones_por_tweet,bombing,suicide,accident,disaster,police,died,years
0,our deed are the reason of this earthquak may ...,1,0,0,1,70,2,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
1,forest fire near la rong sask canada,1,0,0,1,40,1,0.1,0.4,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
target = train_limpio["target"]
target.head(2)

0    1
1    1
Name: target, dtype: int64

In [155]:
VALIDATION_SIZE = .8
TEST_SIZE = .9
df = train_limpio[["text", "target"]]
raw_train_ds, raw_val_ds, raw_test_ds = np.split(df.sample(frac=1), [int(VALIDATION_SIZE*len(df)), int(TEST_SIZE*len(df))])

In [156]:
raw_train_ds.head(2)

,text,target
3977,politician are use fals alleg to attack planne...,0
5322,sammi and todd alway to the rescu may not be i...,0


In [157]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((raw_train_ds["text"].to_numpy(dtype="str"), raw_train_ds["target"].values))
raw_val_ds = tf.data.Dataset.from_tensor_slices((raw_val_ds["text"].to_numpy(dtype="str"), raw_val_ds["target"].values))
raw_test_ds = tf.data.Dataset.from_tensor_slices((raw_test_ds["text"].to_numpy(dtype="str"), raw_test_ds["target"].values))


train_examples_batch, train_labels_batch = next(iter(raw_train_ds.batch(10)))

In [158]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'politician are use fals alleg to attack plannedparenthood amp harm women we arent fool we standwithpp ',
       b'sammi and todd alway to the rescu may not be in uniform but still to the rescu lmao forev kfc fam',
       b'horribl accid man die in wing airplan  watchthevideo ',
       b'video were pick up bodi from water rescuer are search for hundr of migrant in  httptcoawtscucbbv africa news',
       b'propertycasualti insur rate up  in juli after sever month of no movement commerci propertycasu ',
       b'if you cant have the roar of the wave a rainstorm amp some rolling\xc3\x82\xc2\x89\xc3\x82\xc2\x9b httpstcodlvyfvnqe',
       b'rss judg order texa to recogn spous on samesex death certif  ',
       b'white ppl bruis easili where ur bullet wound at  \n\n',
       b'at least  dead a million displac in india flood  india  reliefweb ',
       b'roguewatson noth wrong with that the lethal weapon seri is great yes theyr all great']

In [159]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0])>

In [160]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [161]:
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.8806901 , -0.63843614,  1.1812737 ,  0.5551796 , -0.5470011 ,
        -2.008675  , -0.05108148,  1.1161855 ,  1.0660561 ,  0.5322714 ,
         0.51325893, -0.52694666, -0.4781363 ,  0.5712763 , -1.9952533 ,
         0.7040844 ,  1.5689149 , -1.0832438 , -1.1463627 , -0.37682045],
       [ 0.26734033, -0.38470826,  0.7676315 ,  0.59553486, -0.5820006 ,
        -1.0275128 , -0.2141339 ,  0.4378508 , -0.28285223,  0.36845356,
        -0.07228809,  0.23669232, -0.56938297,  0.27845785, -1.1052439 ,
         0.87956715,  1.2939352 , -0.7076304 , -0.6128581 , -0.14476882],
       [ 0.37912795, -0.20211546, -0.35258245, -0.5629716 ,  0.33592448,
        -0.27964857, -0.17014645,  0.77870774,  0.6412596 ,  0.37272575,
        -0.39419985,  0.45950374, -0.08239779, -0.21659683, -0.78740853,
         1.0758795 , -0.21799651, -0.40427515, -0.97689706,  0.13644128]],
      dtype=float32)>

### Esto es el modelo y, aunque parezca lo más dificil, es lo que más se entiende hasta ahora

In [162]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy', 'binary_crossentropy'])
history = model.fit(raw_train_ds.shuffle(5000).batch(512),
                    epochs=30,
                    validation_data=raw_val_ds.batch(512), verbose=0)
historyMap = {}
historyMap["some model"] = history

In [163]:
model.evaluate(raw_test_ds.batch(512))

2/2 [==============================] - 0s 2ms/step - loss: 0.5177 - accuracy: 0.7493 - binary_crossentropy: 2.8991


[0.5176699161529541, 0.7492997050285339, 2.899094581604004]

## Submission

In [164]:
test_limpio = pd.read_csv("test_limpio_con_stemming.csv",encoding = "ISO-8859-1")
test_limpio.head(2)

,id,keyword,location,text,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,...,he is,insultos_por_tweet,expresiones_por_tweet,bombing,suicide,accident,disaster,police,died,years
0,0,NaN,NaN,just happen a terribl car crash,0,0,1,40,2,-1.00,...,0,0,0,0,0,0,0,0,0,0
1,2,NaN,NaN,heard about earthquak is differ citi stay safe...,0,0,1,70,2,0.25,...,0,0,0,0,0,0,0,0,0,0


In [165]:
predictionsInputsDF = test_limpio["text"]
len(predictionsInputsDF)


3263

In [166]:
predictionsInputsX = tf.data.Dataset.from_tensor_slices((predictionsInputsDF.to_numpy(dtype="str")))
next(iter(predictionsInputsX.batch(10)))

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'just happen a terribl car crash',
       b'heard about earthquak is differ citi stay safe everyon',
       b'there is a forest fire at spot pond gees are flee across the street i cannot save them all',
       b'apocalyps light spokan wildfir',
       b'typhoon soudelor kill  in china and taiwan',
       b'were shakingit an earthquak',
       b'theyd probabl still show more life than arsenal did yesterday eh eh',
       b'hey how are you', b'what a nice hat', b'fuck off'], dtype=object)>

In [167]:
predictionsOutput = model.predict_classes(predictionsInputsX.batch(512))
len(predictionsOutput)

3263

In [168]:
test_limpio["target"] = predictionsOutput

In [169]:
test_limpio[["id", "target"]].head(10)

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [170]:
test_limpio[["id", "target"]].to_csv('NN_predictions.csv', index=False)